# Lab 2: Classification

### Machine Learning 1, September 2015

* The lab exercises should be made in groups of two, three or four people.
* The deadline is October 4th (Sunday) 23:59.
* Assignment should be sent to Philip Versteeg (p.j.j.p.versteeg@uva.nl). The subject line of your email should be "lab\#\_lastname1\_lastname2\_lastname3".
* Put your and your teammates' names in the body of the email
* Attach the .IPYNB (IPython Notebook) file containing your code and answers. Naming of the file follows the same rule as the subject line. For example, if the subject line is "lab01\_Kingma\_Hu", the attached file should be "lab01\_Kingma\_Hu.ipynb". Only use underscores ("\_") to connect names, otherwise the files cannot be parsed.

Notes on implementation:

* For this notebook you need to answer a few theory questions, add them in the Markdown cell's below the question. Note: you can use Latex-style code in here.
* Focus on Part 1 the first week, and Part 2 the second week!
* You should write your code and answers below the questions in this IPython Notebook.
* Among the first lines of your notebook should be "%pylab inline". This imports all required modules, and your plots will appear inline.
* If you have questions outside of the labs, post them on blackboard or email me.
* NOTE: Make sure we can run your notebook / scripts!

$\newcommand{\bx}{\mathbf{x}}$
$\newcommand{\bw}{\mathbf{w}}$
$\newcommand{\bt}{\mathbf{t}}$
$\newcommand{\by}{\mathbf{y}}$
$\newcommand{\bm}{\mathbf{m}}$
$\newcommand{\bb}{\mathbf{b}}$
$\newcommand{\bS}{\mathbf{S}}$
$\newcommand{\ba}{\mathbf{a}}$
$\newcommand{\bz}{\mathbf{z}}$
$\newcommand{\bv}{\mathbf{v}}$
$\newcommand{\bq}{\mathbf{q}}$
$\newcommand{\bp}{\mathbf{p}}$
$\newcommand{\bh}{\mathbf{h}}$
$\newcommand{\bI}{\mathbf{I}}$
$\newcommand{\bX}{\mathbf{X}}$
$\newcommand{\bT}{\mathbf{T}}$
$\newcommand{\bPhi}{\mathbf{\Phi}}$
$\newcommand{\bW}{\mathbf{W}}$
$\newcommand{\bV}{\mathbf{V}}$

In [ ]:
%pylab inline
import gzip, cPickle
from __future__ import division

# Part 1. Multiclass logistic regression

Scenario: you have a friend with one big problem: she's completely blind. You decided to help her: she has a special smartphone for blind people, and you are going to develop a mobile phone app that can do _machine vision_ using the mobile camera: converting a picture (from the camera) to the meaning of the image. You decide to start with an app that can read handwritten digits, i.e. convert an image of handwritten digits to text (e.g. it would enable her to read precious handwritten phone numbers).

A key building block for such an app would be a function `predict_digit(x)` that returns the digit class of an image patch $\bx$. Since hand-coding this function is highly non-trivial, you decide to solve this problem using machine learning, such that the internal parameters of this function are automatically learned using machine learning techniques.

The dataset you're going to use for this is the MNIST handwritten digits dataset (`http://yann.lecun.com/exdb/mnist/`). You can load the data from `mnist.pkl.gz` we provided, using: 

In [ ]:
def load_mnist():
    f = gzip.open('mnist.pkl.gz', 'rb')
    data = cPickle.load(f)
    f.close()
    return data

(x_train, t_train), (x_valid, t_valid), (x_test, t_test) = load_mnist()

The tuples represent train, validation and test sets. The first element (`x_train`, `x_valid`, `x_test`) of each tuple is a $N \times M$ matrix, where $N$ is the number of datapoints and $M = 28^2 = 784$ is the dimensionality of the data. The second element (`t_train`, `t_valid`, `t_test`) of each tuple is the corresponding $N$-dimensional vector of integers, containing the true class labels.

Here's a visualisation of the first 8 digits of the trainingset:

In [ ]:
def plot_digits(data, numcols, shape=(28,28)):
    numdigits = data.shape[0]
    numrows = int(ceil(float(numdigits)/numcols))
    print numcols, numrows, numdigits
    for i in range(numdigits):
        plt.subplot(numrows, numcols, i)
        plt.axis('off')
        plt.imshow(data[i].reshape(shape), interpolation='nearest', cmap='Greys')
    plt.show()
    
plot_digits(x_train[0:8], numcols=4)

In _multiclass_ logistic regression, the conditional probability of class label $j$ given the image $\bx$ for some datapoint is given by:

$ \log p(t = j \;|\; \bx, \bb, \bW) = \log q_j - \log Z$

where $\log q_j = \bw_j^T \bx + b_j$ (the log of the unnormalized probability of the class $j$), and $Z = \sum_k q_k$ is the normalizing factor. $\bw_j$ is the $j$-th column of $\bW$ (a matrix of size $784 \times 10$) corresponding to the class label, $b_j$ is the $j$-th element of $\bb$.

Given an input image, the multiclass logistic regression model first computes the intermediate vector $\log \bq$ (of size $10 \times 1$), using $\log q_j = \bw_j^T \bx + b_j$, containing the unnormalized log-probabilities per class. 

The unnormalized probabilities are then normalized by $Z$ such that $\sum_j p_j = \sum_j \exp(\log p_j) = 1$. This is done by $\log p_j = \log q_j - \log Z$ where $Z = \sum_j \exp(\log q_j)$. This is known as the _softmax_ transformation, and is also used as a last layer of many classifcation neural network models, to ensure that the output of the network is a normalized distribution, regardless of the values of second-to-last layer ($\log \bq$)

Warning: when computing $\log Z$, you are likely to encounter numerical problems. Save yourself countless hours of debugging and learn the [log-sum-exp trick](https://hips.seas.harvard.edu/blog/2013/01/09/computing-log-sum-exp/ "Title").

The network's output $\log \bp$ of size $10 \times 1$ then contains the conditional log-probabilities $\log p(t = j \;|\; \bx, \bb, \bW)$ for each digit class $j$. In summary, the computations are done in this order:

$\bx \rightarrow \log \bq \rightarrow Z \rightarrow \log \bp$

Given some dataset with $N$ independent, identically distributed datapoints, the log-likelihood is given by:

$ \mathcal{L}(\bb, \bW) = \sum_{n=1}^N \mathcal{L}^{(n)}$

where we use $\mathcal{L}^{(n)}$ to denote the partial log-likelihood evaluated over a single datapoint. It is important to see that the log-probability of the class label $t^{(n)}$ given the image, is given by the $t^{(n)}$-th element of the network's output $\log \bp$, denoted by $\log p_{t^{(n)}}$:

$\mathcal{L}^{(n)} = \log p(t = t^{(n)} \;|\; \bx = \bx^{(n)}, \bb, \bW) = \log p_{t^{(n)}} = \log q_{t^{(n)}} - \log Z^{(n)}$

where $\bx^{(n)}$ and $t^{(n)}$ are the input (image) and class label (integer) of the $n$-th datapoint, and $Z^{(n)}$ is the normalizing constant for the distribution over $t^{(n)}$.


## 1.1 Gradient-based stochastic optimization

### 1.1.1 Derive gradient equations (20 points)

Derive the equations for computing the (first) partial derivatives of the log-likelihood w.r.t. all the parameters, evaluated at a _single_ datapoint $n$.

You should start deriving the equations for $\frac{\partial \mathcal{L}^{(n)}}{\partial \log q_j}$ for each $j$. For clarity, we'll use the shorthand $\delta^q_j = \frac{\partial \mathcal{L}^{(n)}}{\partial \log q_j}$.

For $j = t^{(n)}$:
$
\delta^q_j
= \frac{\partial \mathcal{L}^{(n)}}{\partial \log p_j}
\frac{\partial \log p_j}{\partial \log q_j}
+ \frac{\partial \mathcal{L}^{(n)}}{\partial \log Z}
\frac{\partial \log Z}{\partial Z} 
\frac{\partial Z}{\partial \log q_j} 
= 1 \cdot 1 - \frac{\partial \log Z}{\partial Z} 
\frac{\partial Z}{\partial \log q_j}
= 1 - \frac{\partial \log Z}{\partial Z} 
\frac{\partial Z}{\partial \log q_j}
$

For $j \neq t^{(n)}$:
$
\delta^q_j
= \frac{\partial \mathcal{L}^{(n)}}{\partial \log Z}
\frac{\partial \log Z}{\partial Z} 
\frac{\partial Z}{\partial \log q_j} 
= - \frac{\partial \log Z}{\partial Z} 
\frac{\partial Z}{\partial \log q_j}
$

Complete the above derivations for $\delta^q_j$ by furtherly developing $\frac{\partial \log Z}{\partial Z}$ and $\frac{\partial Z}{\partial \log q_j}$. Both are quite simple. For these it doesn't matter whether $j = t^{(n)}$ or not.

Given your equations for computing the gradients $\delta^q_j$ it should be quite straightforward to derive the equations for the gradients of the parameters of the model, $\frac{\partial \mathcal{L}^{(n)}}{\partial W_{ij}}$ and $\frac{\partial \mathcal{L}^{(n)}}{\partial b_j}$. The gradients for the biases $\bb$ are given by:

$
\frac{\partial \mathcal{L}^{(n)}}{\partial b_j}
= \frac{\partial \mathcal{L}^{(n)}}{\partial \log q_j}
\frac{\partial \log q_j}{\partial b_j}
= \delta^q_j
\cdot 1
= \delta^q_j
$

The equation above gives the derivative of $\mathcal{L}^{(n)}$ w.r.t. a single element of $\bb$, so the vector $\nabla_\bb \mathcal{L}^{(n)}$ with all derivatives of $\mathcal{L}^{(n)}$ w.r.t. the bias parameters $\bb$ is: 

$
\nabla_\bb \mathcal{L}^{(n)} = \mathbf{\delta}^q
$

where $\mathbf{\delta}^q$ denotes the vector of size $10 \times 1$ with elements $\mathbf{\delta}_j^q$.

The (not fully developed) equation for computing the derivative of $\mathcal{L}^{(n)}$ w.r.t. a single element $W_{ij}$ of $\bW$ is:

$
\frac{\partial \mathcal{L}^{(n)}}{\partial W_{ij}} =
\frac{\partial \mathcal{L}^{(n)}}{\partial \log q_j}
\frac{\partial \log q_j}{\partial W_{ij}}
= \mathbf{\delta}_j^q
\frac{\partial \log q_j}{\partial W_{ij}}
$

What is $\frac{\partial \log q_j}{\partial W_{ij}}$? Complete the equation above.

If you want, you can give the resulting equation in vector format ($\nabla_{\bw_j} \mathcal{L}^{(n)} = ...$), like we did for $\nabla_\bb \mathcal{L}^{(n)}$.

#####Answer: 

\begin{align*}
\frac{\partial \log Z}{\partial Z} &= 1/Z\\
\frac{\partial Z}{\partial \log q_j} &= q_j
\end{align*}

We get $\frac{\partial \log q_j}{ \partial W_{ij}} = x_i$ so we get

\begin{align*}
j=t(n): \; \frac{\partial L^{(n)}}{\partial W_{ij}}&=x_i-\frac{q_j x_i}{Z}\\
j\neq t(n): \; \frac{\partial L^{(n)}}{\partial W_{ij}}&=\frac{-q_j x_i}{Z}
\end{align*}
Alternatively:
$$
\nabla_{w_j} L^{(n)} = \delta^q_j x
$$

### 1.1.2 Implement gradient computations (10 points)

Implement the gradient calculations you derived in the previous question. Write a function `logreg_gradient(x, t, w, b)` that returns the gradients $\nabla_{\bw_j} \mathcal{L}^{(n)}$ (for each $j$) and $\nabla_{\bb} \mathcal{L}^{(n)}$, i.e. the first partial derivatives of the log-likelihood w.r.t. the parameters $\bW$ and $\bb$, evaluated at a single datapoint (`x`, `t`).
The computation will contain roughly the following intermediate variables:

$
\log \bq \rightarrow Z \rightarrow \log \bp\,,\, \mathbf{\delta}^q
$

followed by computation of the gradient vectors $\nabla_{\bw_j} \mathcal{L}^{(n)}$ (contained in a $784 \times 10$ matrix) and $\nabla_{\bb} \mathcal{L}^{(n)}$ (a $10 \times 1$ vector).


In [ ]:
def logreg_gradient(x, t, w, b):
    """
    Returns the gradients of L with respect to all $w_j$ and $b$.
    - x (784 x 1)
    - t (10 x 1)
    - w (784 x 10)
    - b (10 x 1)
    """    
    # Vector with all log q's
    log_Q = w.T.dot(x) + b
    
    # Variation on the log-sum-exp trick:
    # exp(-a) exp(log_Q)           exp(log_Q - a)
    # ———————————————————————   =  ——————————————
    # exp(-a) sum(exp(log_Q))      sum(exp(log_Q - a))
    #
    a = np.max(log_Q)
    deltas = -exp(log_Q - a) / sum(exp(log_Q - a, dtype=np.longdouble))
    deltas[t] = 1 + deltas[t] # the case t_n = j
    grad_b = deltas
    
    grad_w = np.zeros([784,10])
    for j in range(10):
        grad_w[:,j] = deltas[j]*x

    return (grad_w, grad_b)




### 1.1.3 Stochastic gradient descent (10 points)

Write a function `sgd_iter(x_train, t_train, w, b)` that performs one iteration of stochastic gradient descent (SGD), and returns the new weights. It should go through the trainingset once in randomized order, call `logreg_gradient(x, t, w, b)` for each datapoint to get the gradients, and update the parameters using a small learning rate (e.g. `1E-4`). Note that in this case we're maximizing the likelihood function, so we should actually performing gradient ___ascent___... For more information about SGD, see Bishop 5.2.4 or an online source (i.e. https://en.wikipedia.org/wiki/Stochastic_gradient_descent)

In [ ]:
def sgd_iter( x_train, t_train, w, b):

    # Learning rate
    eta = 1e-4
    
    indices = arange(len(x_train))
    random.shuffle(indices)
    for n in indices:
        grad_w, grad_b = logreg_gradient( x_train[n], t_train[n], w, b)
        b = b + eta * grad_b
        w = w + eta * grad_w
    
    return w, b

## 1.2. Train

### 1.2.1 Train (10 points)
Perform a handful of training iterations through the trainingset. Plot (in one graph) the conditional log-probability of the trainingset and validation set after each iteration.


In [ ]:
def data_likelihood(X, T, w, b):
    
    N = X.shape[0]

    # Vector with all log_q's 
    log_Q = X.dot(w) + b

    # Log likelihoods of the training data
    log_Q_T = log_Q[arange(N), T]

    # Maximum element from every row of log_Q
    # but repeated 10 times to be able to substract it from log_Q
    max_indices = np.argmax(log_Q, axis=1)
    max_Qs = (log_Q[arange(N), max_indices]).repeat(10).reshape(N,10)
    log_Z = log_Q[arange(N), max_indices] + sum(exp(log_Q - max_Qs), axis=1)

    return sum(log_Q_T - log_Z)

In [ ]:
w = np.random.randn(784, 10)
b = np.random.randn(10)
num_iterations = 20
Lvec_train = np.zeros(num_iterations)
Lvec_valid = np.zeros(num_iterations)

for i in range(num_iterations):
    print i
    
    w, b = sgd_iter( x_train, t_train, w, b)
    
    L_train = data_likelihood(x_train, t_train, w, b) 
    L_valid = data_likelihood(x_valid, t_valid, w, b)
    
    Lvec_train[i] = L_train
    Lvec_valid[i] = L_valid
    
plot(range(num_iterations),Lvec_train)
plot(range(num_iterations),Lvec_valid)
show()

### 1.2.2 Visualize weights (10 points)
Visualize the resulting parameters $\bW$ after a few iterations through the training set, by treating each column of $\bW$ as an image. If you want, you can use or edit the `plot_digits(...)` above.

In [ ]:
plot_digits( w.T, 5)

### 1.2.3. Visualize the 8 hardest and 8 easiest digits (10 points)
Visualize the 8 digits in the validation set with the highest probability of the true class label under the model.
Also plot the 8 digits that were assigned the lowest probability.
Ask yourself if these results make sense.

In [ ]:
L_n_valid = zeros(len(x_valid))

for n in range(len(x_valid)):
    log_qs = np.dot(w.T, x_valid[n]) + b
    kmax = np.argmax(log_qs)
    log_q_tn = log_qs[t_valid[n]]
    log_Z = log_qs[kmax] + np.sum(np.exp(log_qs - log_qs[kmax]))
    L_n_valid[n] = log_q_tn - log_Z

easiest = L_n_valid.argsort()[-8:][::-1]
hardest = L_n_valid.argsort()[:8]

plot_digits(x_valid[easiest], numcols=4)
plot_digits(x_valid[hardest], numcols=4)

# Part 2. Multilayer perceptron


You discover that the predictions by the logistic regression classifier are not good enough for your application: the model is too simple. You want to increase the accuracy of your predictions by using a better model. For this purpose, you're going to use a multilayer perceptron (MLP), a simple kind of neural network. The perceptron wil have a single hidden layer $\bh$ with $L$ elements. The parameters of the model are $\bV$ (connections between input $\bx$ and hidden layer $\bh$), $\ba$ (the biases/intercepts of $\bh$), $\bW$ (connections between $\bh$ and $\log q$) and $\bb$ (the biases/intercepts of $\log q$.

The conditional probability of the class label $j$ is given by:

$\log p(t = j \;|\; \bx, \bb, \bW) = \log q_j - \log Z$

where $q_j$ are again the unnormalized probabilities per class, and $Z = \sum_j q_j$ is again the probability normalizing factor. Each $q_j$ is computed using:

$\log q_j = \bw_j^T \bh + b_j$

where $\bh$ is a $L \times 1$ vector with the hidden layer activations (of a hidden layer with size $L$), and $\bw_j$ is the $j$-th column of $\bW$ (a $L \times 10$ matrix). Each element of the hidden layer is computed from the input vector $\bx$ using:

$h_j = \sigma(\bv_j^T \bx + a_j)$

where $\bv_j$ is the $j$-th column of $\bV$ (a $784 \times L$ matrix), $a_j$ is the $j$-th element of $\ba$, and $\sigma(.)$ is the so-called sigmoid activation function, defined by:

$\sigma(x) = \frac{1}{1 + \exp(-x)}$

Note that this model is almost equal to the multiclass logistic regression model, but with an extra 'hidden layer' $\bh$. The activations of this hidden layer can be viewed as features computed from the input, where the feature transformation ($\bV$ and $\ba$) is learned.

## 2.1 Derive gradient equations (20 points)

State (shortly) why $\nabla_{\bb} \mathcal{L}^{(n)}$ is equal to the earlier (multiclass logistic regression) case, and why $\nabla_{\bw_j} \mathcal{L}^{(n)}$ is almost equal to the earlier case.

Like in multiclass logistic regression, you should use intermediate variables $\mathbf{\delta}_j^q$. In addition, you should use intermediate variables $\mathbf{\delta}_j^h = \frac{\partial \mathcal{L}^{(n)}}{\partial h_j}$.

Given an input image, roughly the following intermediate variables should be computed:

$
\log \bq \rightarrow Z \rightarrow \log \bp \rightarrow \mathbf{\delta}^q \rightarrow \mathbf{\delta}^h
$

where $\mathbf{\delta}_j^h = \frac{\partial \mathcal{L}^{(n)}}{\partial \bh_j}$.

Give the equations for computing $\mathbf{\delta}^h$, and for computing the derivatives of $\mathcal{L}^{(n)}$ w.r.t. $\bW$, $\bb$, $\bV$ and $\ba$. 

You can use the convenient fact that $\frac{\partial}{\partial x} \sigma(x) = \sigma(x) (1 - \sigma(x))$.

#####Answer: 

Well, the derivation of $\nabla_{\mathbf v} \mathcal{L}^{(n)}$ remains exactly the same since $\frac{\partial \log q_i}{\partial b_i} = 1$:
\begin{align}
\nabla_{{\bf b}}\mathcal{L}^{(n)} = \delta^q
\end{align}

For $\nabla_{{\bf w}_j} \mathcal{L}^{(n)}$, things are slightly different,
\begin{align}
\frac{\partial \mathcal{L}^{(n)}}{\partial W_{ij}} 
	&= \frac{\partial \mathcal{L}^{(n)}}{\partial \log q_j} \frac{\partial \log q_j}{\partial W_{ij}}
	= \delta_j^q \cdot \frac{\partial}{\partial W_{ij}} ({\bf w}_j^T {\bf h} + b_j)
	= \delta_j^q \cdot h_i.
\end{align}
Hence
\begin{align}
	\nabla_{{\bf w}_j} \mathcal{L}^{(n)} = \delta_j^q \cdot {\bf h},
\end{align}
and the only difference with the former case is that we replaced ${\bf x}$ by $\bf h$.

--------
The derivation of $\delta^h$ is even trickier:
\begin{align}
\delta^h_j 
	&= \frac{\partial{\mathcal{L}^{(n)}}}{\partial h_j}
	= \frac{\partial}{\partial h_j} \log q_k - \log Z \\
	&= \frac{\partial}{\partial h_j} {\bf w}_k^T {\bf h} + b_k 
		- \frac{1}{Z} \sum_i \frac{\partial}{\partial h_j} \exp \log q_i \\
	&= W_{jk} - \frac{1}{Z} \sum_i q_i \frac{\partial}{\partial h_j} \log q_i \\
	&= W_{jk} - \frac{1}{Z} \sum_i q_i W_{ji}
\end{align}

That gives
$
	\delta^h = {\bf w}^k - \frac{1}{Z} {\bf W} \bf{q}
$


\begin{align}
\frac{\partial \mathcal L^{(n)}}{\partial a_j}
    &= \frac{\partial \mathcal L^{(n)}}{\partial h_j} \frac{\partial h_j}{\partial a_j}\\
    &= \delta_j^h  \cdot \sigma'(\hat h_j) \cdot  \frac{\partial}{\partial a_j} ({\bf v}_j^T{\bf x} + a_j)\\
    &= \delta_j^h  \cdot \sigma'(\hat h_j)
\end{align}

Hence
\begin{align}
\nabla_{\bf a} \mathcal L^{(n)} 
    = \Bigl(\delta_1^h \; \sigma'(\hat h_1), 
    \quad \delta_2^h \; \sigma'(\hat h_2), \; \dots\Bigr)^T 
\end{align}

Some more abbreviations:
\begin{align}
\hat h_j := {\bf v}_j^T {\bf x} + a_j, 
\qquad \text{and} \qquad \sigma'(x) := \frac{\partial}{\partial x}\sigma(x) = \sigma(x)(1-\sigma(x))
\end{align}
With these definitions $h_j = \sigma(\hat h_j)$ and we get
\begin{align}
\frac{\partial \mathcal L^{(n)}}{\partial V_{ij}}
	&= \frac{\partial \mathcal L^{(n)}}{\partial h_j} \frac{\partial}{\partial V_{ij}} \sigma( \hat h_j)\\
	&= \delta_j^h \cdot \sigma'(\hat h_j) \cdot \frac{\partial}{\partial V_{ij}} ( {\bf v}_j^T {\bf x} + a_j )\\
	&= \delta_j^h \cdot \sigma'(\hat h_j) \cdot x_i
\end{align}
Hence
\begin{align}
	\nabla_{{\bf v}_j} \mathcal L^{(n)} 
    = \delta_j^h \cdot \sigma(\hat h_j)(1-\sigma(\hat h_j)) \cdot {\bf x}
\end{align}

## 2.2 MAP optimization (10 points)

You derived equations for finding the _maximum likelihood_ solution of the parameters. Explain, in a few sentences, how you could extend this approach so that it optimizes towards a _maximum a posteriori_ (MAP) solution of the parameters, with a Gaussian prior on the parameters. 

#####Answer: 

If we placed a Gaussian prior on the parameters, we could look for the optimal parameters by optimising the posterior distribution
\begin{align}
p(\theta | D) \propto p(t = j | {\bf x}, \theta) \cdot p(\theta | D)
\end{align}
The latter distribution would be a normal distribution, depending on some hyperparameters. If we were to work out the details we would have for the log likelihood
\begin{align}
\log p_j = \log q_j - \log Z + \log q(\theta |D).
\end{align}

## 2.3. Implement and train a MLP (15 points)

Implement a MLP model with a single hidden layer, and code to train the model.

In [ ]:
def sigmoid(x):
      return 1/(1+exp(-x, dtype=np.longdouble))

In [ ]:
def mlp(X,T,v,a,w,b):
    N = X.shape[0]
    
    # Compute hidden layer activations
    h = 1/(1+exp(-(X.dot(v)+a)))
    
    # Matrix with all log-likelihoods
    log_Q = h.dot(w)+b
    
    # Log-likelihoods of the training data
    log_Q_T = log_Q[arange(N), T]
    
    # Maximum element from every row of log_Q
    # but repeated 10 times to be able to substract it from log_Q
    max_indices = np.argmax(log_Q, axis=1)
    max_Qs = (log_Q[arange(N), max_indices]).repeat(10).reshape(N,10)
    log_Z = log_Q[arange(N), max_indices] + sum(exp(log_Q - max_Qs), axis=1)
    
    return sum(log_Q_T - log_Z)

def mlp_gradient(x,t,v,a,w,b):
    """
    Returns the gradients of L with respect to $v$, $a$, $w$ and $b$.
    - x (784 x 1)
    - t (10 x 1)
    - v (784 x L)
    - a (L x 1)
    - w (L x 10)
    - b (10 x 1)
    """
    l = w.shape[0]
    # Compute hidden layer activations
    h = sigmoid(v.T.dot(x)+a)
    # Very weird bug, sometime h returns as (L,)-array, sometimes as (1,L)-matrix, workaround
    h =  matrix(h).A1
    # Matrix with all log-likelihoods
    log_Q = w.T.dot(h)+b
    
    A = log_Q[ argmax(log_Q) ]
    deltaqs = -exp(log_Q - A)/sum(exp(log_Q - A, dtype=np.longdouble))
    deltaqs[t] = 1+deltaqs[t] # the case t_n = j
    grad_b = deltaqs
    
    grad_w = np.zeros([l,10])
    for j in range(10):
        grad_w[:,j] = deltaqs[j]*h

    # this is not correct, but not enough time to implement correct delta
    deltahs = deltaqs.dot(w.T)
    # sigmoid' = sigmoid*(1-sigmoid)
    grad_a = deltahs*sigmoid(h)*(1-sigmoid(h))
    grad_v = matrix(x).T.dot(matrix(grad_a))
    
    # Cast a and b back to array before returning
    return (grad_v,grad_a,grad_w,grad_b)

def mlp_train(x,t,v,a,w,b):
    # Learning rate
    eta = 1e-4
    
    indices = arange(len(x_train))
    random.shuffle(indices)
    for n in indices:
        grad_v,grad_a,grad_w,grad_b = mlp_gradient(x[n],t[n],v,a,w,b)
        v = v + eta * grad_v
        a = a + eta * grad_a
        w = w + eta * grad_w
        b = b + eta * grad_b
    
    return v, a, w, b

def mlp_ratio_misclass(X,T,v,a,w,b):
    N = len(T)
    # Compute hidden layer activations
    h = sigmoid(X.dot(v)+a)
    # Vector with all log-likelihoods
    log_Q = h.dot(w)+b
    return (N-sum(argmax(log_Q,1).A1 == T))/N

### 2.3.1. Less than 250 misclassifications on the test set (10 bonus points)

You receive an additional 10 bonus points if you manage to train a model with very high accuracy: at most 2.5% misclasified digits on the test set. Note that the test set contains 10000 digits, so you model should misclassify at most 250 digits. This should be achievable with a MLP model with one hidden layer. See results of various models at : `http://yann.lecun.com/exdb/mnist/index.html`. To reach such a low accuracy, you probably need to have a very high $L$ (many hidden units), probably $L > 200$, and apply a strong Gaussian prior on the weights. In this case you are allowed to use the validation set for training.
You are allowed to add additional layers, and use convolutional networks, although that is probably not required to reach 2.5% misclassifications.

In [ ]:
l = 200
V = np.random.randn(784,l)
a = np.random.randn(l)
W = np.random.randn(l,10)
b = np.random.randn(10)

misclass = 1
while misclass > 0.024:
    V,a,W,b = mlp_train(x_train,t_train,V,a,W,b)
    misclass = mlp_ratio_misclass(x_valid,t_valid,V,a,W,b)
    print misclass
# Calculation takes ages, does improve but very slooowwww...

print mlp_ratio_misclass(x_test,t_test,V,a,W,b)